# Семинар 5. Групповая обработка и слияние наборов данных в Pandas

На этом семинаре мы поработаем с данными о продажах глобального интернет-магазина, реализующего офисные товары и технику. Цель занятия - научиться использовать групповую обработку в Pandas и соединять таблицы данных.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Загрузим данные:

In [2]:
orders = pd.read_csv('../datasets/global_superstore/orders.zip', encoding='utf-8')
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   row_id          51290 non-null  int64  
 1   order_id        51290 non-null  object 
 2   order_date      51290 non-null  object 
 3   ship_date       51290 non-null  object 
 4   ship_mode       51290 non-null  object 
 5   customer_id     51290 non-null  object 
 6   customer_name   51290 non-null  object 
 7   segment         51290 non-null  object 
 8   postal_code     9994 non-null   float64
 9   city            51290 non-null  object 
 10  state           51290 non-null  object 
 11  country         51290 non-null  object 
 12  region          51290 non-null  object 
 13  market          51290 non-null  object 
 14  product_id      51290 non-null  object 
 15  category        51290 non-null  object 
 16  sub_category    51290 non-null  object 
 17  product_name    51290 non-null 

При загрузке со стандартными параметрами появилось несколько проблем:

Столбец почтового индекса (postal_code) распознано как числовой. Однако в некоторых странах почтовый индекс может содержать буквы и другие символы.

Даты загрузились как текст

Добавим спецификацию форматов данных в команду для загрузки:

In [4]:
orders = pd.read_csv('../datasets/global_superstore/orders.zip', encoding='utf-8',
                    parse_dates=['order_date', 'ship_date'],
                    dtype={'postal_code' : str})
orders.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120,Oklahoma City,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.980,2,0.0,62.1544,40.77,High
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          51290 non-null  int64         
 1   order_id        51290 non-null  object        
 2   order_date      51290 non-null  datetime64[ns]
 3   ship_date       51290 non-null  datetime64[ns]
 4   ship_mode       51290 non-null  object        
 5   customer_id     51290 non-null  object        
 6   customer_name   51290 non-null  object        
 7   segment         51290 non-null  object        
 8   postal_code     9994 non-null   object        
 9   city            51290 non-null  object        
 10  state           51290 non-null  object        
 11  country         51290 non-null  object        
 12  region          51290 non-null  object        
 13  market          51290 non-null  object        
 14  product_id      51290 non-null  object        
 15  ca

Теперь данные загрузились правильно. Даты стали датами, а postal_code теперь - текстовое поле. Количество пропущенных значений не изменилось, в данных действительно во многих случаях индекс не указан.

При загрузке из текстовых файлов может возникать множество проблем из-за ошибок в данных или несоответствия форматов. При необходимости обратитесь к ресурсам:

справка Pandas https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
Раздел 6.1 в книге Уэса Маккинни Python и анализ данных (см. ресурсы в LMS)

Словарь данных:

row_id - код строки заказа (уникальный)

order_id - код заказа

order_date - дата размещения заказа

ship_date - дата отгрузки заказа (пусто, если еще не отгружен)

ship_mode - класс отправки

customer_id - код клиента

customer_name - имя клиента

segment - рыночный сегмент клиента

postal_code - почтовый индекс получателя

city - город получателя

state - штат получателя

country - страна получателя

region - регион получателя

market - рынок, к которому относится получатель

product_id - код товара

category - товарная категория

sub_category - товарная подкатегория

product_name - наименование товара

sales - выручка

quantity - количество единиц товара

discount - скидка

profit - валовая прибыль

shipping_cost - затраты на доставку

order_priority - приоритет заказа

Данные представляют собой строки заказов, выполненных компанией. Каждая строка имеет уникальный номер row_id и содержит данные о том, кто, что и когда заказал, когда эта позиция была отправлена, адрес получателя. Также количество единиц, выручку, прибыль и затраты на доставку по строке.

В одном заказе может быть несколько позиций. В этом случае в таблице будет несколько строк с одинаковым номером заказа:

In [6]:
orders.query('order_id == "AE-2015-GH4665138-42351"')

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
27486,43926,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,TEC-PH-5253,Technology,Phones,"Motorola Headset, VoIP",95.796,4,0.7,-156.564,6.73,Medium
29298,43924,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-4078,Office Supplies,Storage,"Eldon Lockers, Blue",59.373,1,0.7,-118.767,5.83,Medium
33521,43922,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-AR-3463,Office Supplies,Art,"BIC Pencil Sharpener, Water Color",77.256,8,0.7,-82.584,4.08,Medium
40033,43925,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-ST-5693,Office Supplies,Storage,"Rogers File Cart, Single Width",42.489,1,0.7,-62.331,2.24,Medium
49469,43923,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3246,Office Supplies,Binders,"Avery Binder Covers, Clear",3.159,1,0.7,-4.971,1.25,Medium
49470,43927,AE-2015-GH4665138-42351,2015-12-13,2015-12-19,Standard Class,GH-4665138,Greg Hansen,Consumer,NaN,Ras al Khaymah,...,OFF-BI-3249,Office Supplies,Binders,"Avery Binder Covers, Recycled",3.429,1,0.7,-3.891,1.25,Medium


# Задание 1

Во всех заданиях необходимо округлить результат до разумного количества десятичных знаков. Крупные суммы выводите в подходящих единицах (тысячи или млн долларов)

## 1.1. Сколько составляет оборот по каждой товарной категории?

(выведите ответ в млн долларов с округлением до 1 десятичного знака).

In [7]:
sales = orders[['category', 'sales']].groupby('category').sum()
sales["sales"] = sales["sales"] / 1_000_000
sales.round(1)

,sales
category,
Furniture,4.1
Office Supplies,3.8
Technology,4.7


## 1.2. Сколько товаров представлено в каждой товарной категории?

(попробуйте реализовать двумя способами: только с помощью группировки/определения размера группы, и через агрегирующую функцию nunique() - число уникальных значений.

1 способ:

In [8]:
orders.groupby('category').size()

category
Furniture           9860
Office Supplies    31289
Technology         10141
dtype: int64

2 способ:

In [9]:
orders.groupby('category')['row_id'].nunique()

category
Furniture           9860
Office Supplies    31289
Technology         10141
Name: row_id, dtype: int64

## 1.3. Составьте рейтинг стран на основе прибыли от продаж.

In [10]:
countries = orders[['country', 'profit']].groupby('country').sum();
countries.sort_values('profit', ascending=False)

,profit
country,
United States,286397.0217
China,150683.0850
India,129050.5950
United Kingdom,111900.1500
France,109029.0030
...,...
Pakistan,-22446.6480
Honduras,-29482.3714
Netherlands,-41070.0750


## 1.4. Выведите 3 самых популярных (по количеству заказов) товара на каждом рынке.

In [11]:
products = orders[['market', 'row_id', 'product_name']]
products_by_markets = products.groupby('market')
for name, data in products_by_markets:
    print(name)
    print(data.groupby('product_name').size().sort_values(ascending=False).head(3).to_string(), '\n')

Africa
product_name
Ibico Index Tab, Clear                   17
Stanley Pencil Sharpener, Water Color    16
Smead File Cart, Single Width            16 

Asia Pacific
product_name
Cardinal Index Tab, Clear                31
Stockwell Paper Clips, Assorted Sizes    30
Smead File Cart, Single Width            26 

Europe
product_name
Eldon File Cart, Single Width            37
Stanley Pencil Sharpener, Water Color    31
Sanford Pencil Sharpener, Water Color    31 

LATAM
product_name
Acco Index Tab, Clear                  22
Cardinal Index Tab, Clear              20
Stockwell Push Pins, Assorted Sizes    19 

USCA
product_name
Staples                       227
Avery Non-Stick Binders        20
KI Adjustable-Height Table     18 



## 1.5. Сравните рынки по характеристикам заказов на них:
- Средний и медианный чек (сумма заказа)
- Среднее и медианное количество позиций в заказе
- Средняя и медианная прибыль по заказу
- Средняя и медианная стоимость доставки по заказу
- Среднее и медианное отношение стоимости доставки заказа к его сумме

Обратите внимание, что анализировать нужно именно характеристики заказов. В таблице содержатся строки заказов, сначала на их основе нужно посчитать характеристики заказов.

In [12]:
markets = orders.groupby('market')
for name, data in markets:
    print(name)
    grouped_orders = data.groupby('order_id')
    summed_values = grouped_orders.sum()[['sales', 'quantity', 'profit', 'shipping_cost']]
    summed_values['shipcost_to_sales'] = summed_values.shipping_cost / summed_values.sales
    print(summed_values.agg(['mean', 'median']))
    print('\n')
    

Africa
             sales  quantity     profit  shipping_cost  shipcost_to_sales
mean    342.558222  4.617133  38.842496       38.82844           0.123446
median  129.840000  3.000000  18.480000       12.16000           0.102468


Asia Pacific
             sales  quantity     profit  shipping_cost  shipcost_to_sales
mean    570.996931  6.863983  56.945768      61.844373           0.116033
median  239.481300  5.000000  16.530000      22.725000           0.096039


Europe
             sales  quantity   profit  shipping_cost  shipcost_to_sales
mean    546.704845  6.971395  74.7633      58.025118           0.112393
median  256.140000  5.000000  30.3720      24.060000           0.093352


LATAM
             sales  quantity    profit  shipping_cost  shipcost_to_sales
mean    420.801938  7.489502  43.08777      45.689272           0.113586
median  193.620000  6.000000  17.40800      18.122500           0.094330


USCA
             sales  quantity     profit  shipping_cost  shipcost_to_sales
m

## 1.6. Рейтинг по относительной доле затрат на доставку

Рассчитайте для каждой строки (позиции) заказа долю затрат на доставку к выручке по этой строке. Для каждого рынка составьте рейтинг из 5 самых дорогих по стоимости доставке позиций. Чтобы их найти, посчитайте для каждой строки отличие доли затрат на доставку этой позиции к средней доле затрат на доставку для этого региона, товарной категории и класса доставки.

Подсказка: для подсчета отличий по группам используйте transform().

In [13]:
orders['share_delcost_sales'] = orders.shipping_cost / orders.sales
orders['share_delcost_sales_mean'] = (orders
                                          .groupby(['region', 'category', 'ship_mode'])['share_delcost_sales']
                                          .transform('mean'))
orders['dif_share'] = orders.share_delcost_sales - orders.share_delcost_sales_mean
markets = orders.groupby('market')
for name, data in markets:
    print(name)
    print(data[['row_id', 'dif_share']].sort_values('dif_share', ascending=False).head(5), '\n')

Africa
       row_id  dif_share
50259   44693   0.817995
51126   48373   0.805216
50715   45002   0.774929
51218   48012   0.769144
50702   43760   0.745916 

Asia Pacific
       row_id  dif_share
50719   41882   0.621549
51063   42076   0.593771
50706   44680   0.562004
50538   41770   0.550212
50159   43791   0.485207 

Europe
       row_id  dif_share
51078   46146   0.465739
9137    15114   0.355445
46963   43031   0.349086
13334   13784   0.349047
49772   19599   0.320459 

LATAM
       row_id  dif_share
46379     351   0.643021
51230    8277   0.571673
51015     438   0.502331
44570    8688   0.479245
48339   10099   0.454292 

USCA
       row_id  dif_share
51260   35398   2.117484
45391   40589   1.678808
33493   39955   1.110651
13551   32982   1.072776
26588   36008   1.045980 



## 1.7. Сравните динамику продаж по годам различных товарных подкатегорий категории Technology.

Попробуйте два способа: через pivot_table() и через unstack().

Подсказка: компоненты даты - например, год, месяц можно получить на основе столбца с датами с помощью атрибутов dt.year, dt.month, dt.day:

1 способ:

In [14]:
orders
orders['order_year'] = pd.to_datetime(orders['order_date']).dt.year
technology = orders.loc[orders['category'] == 'Technology']
technology.pivot_table(index=['order_year'], columns=['sub_category'], aggfunc="size")

sub_category,Accessories,Copiers,Machines,Phones
order_year,,,,
2012,537,353,279,621
2013,665,504,320,721
2014,840,584,369,893
2015,1033,782,518,1122


2 способ

In [15]:
technology.groupby(['order_year', 'sub_category']).size().unstack()

sub_category,Accessories,Copiers,Machines,Phones
order_year,,,,
2012,537,353,279,621
2013,665,504,320,721
2014,840,584,369,893
2015,1033,782,518,1122


## 1.8. Сравните динамику продаж по годам товаров брендов Apple и Samsung

In [16]:
samsung_apple = orders.copy()
samsung_apple = samsung_apple[samsung_apple['product_name'].str.lower().str.contains('apple') | samsung_apple['product_name'].str.lower().str.contains('samsung')]
samsung_apple['brand_name'] = np.where(samsung_apple['product_name'].str.lower().str.contains('samsung'), 'Samsung', 'Apple',)
samsung_apple.groupby(['order_year', 'brand_name']).size().unstack()

brand_name,Apple,Samsung
order_year,,
2012,104,96
2013,120,140
2014,169,128
2015,173,198


## 1.9. Составьте рейтинг стран по прибыли.

В списке должны быть только страны, в которые отправлено более 500 заказов.

In [17]:
(orders
     .groupby('country')
     .filter(lambda x: len(x) > 500)
     .groupby('country')
     .sum()['profit']
     .sort_values(ascending=False))

country
United States         286397.02170
China                 150683.08500
India                 129050.59500
United Kingdom        111900.15000
France                109029.00300
Germany               107190.32550
Australia             103907.43300
Mexico                102453.58552
Spain                  54390.12000
El Salvador            42023.24320
Cuba                   38889.21548
Nicaragua              33401.44456
Brazil                 29686.89896
Guatemala              27944.68676
Iran                   26856.24000
Italy                  19985.67600
New Zealand            16600.28400
Indonesia              15608.67790
Dominican Republic     -7613.49872
Philippines           -16128.22500
Honduras              -29482.37140
Nigeria               -80750.71800
Turkey                -98447.23200
Name: profit, dtype: float64

# Задание 2

Дополним данные сведениями о том, какие заказы были возвращены.

In [18]:
returns = pd.read_csv('../datasets/global_superstore/returns.csv')
returns.head()

,returned,order_id,region
0,True,CA-2012-SA20830140-41210,Central US
1,True,IN-2012-PB19210127-41259,Eastern Asia
2,True,CA-2012-SC20095140-41174,Central US
3,True,IN-2015-JH158207-42140,Oceania
4,True,IN-2014-LC168857-41747,Oceania


In [19]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1079 entries, 0 to 1078
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   returned  1079 non-null   bool  
 1   order_id  1079 non-null   object
 2   region    1079 non-null   object
dtypes: bool(1), object(2)
memory usage: 18.0+ KB


В таблице returns содержатся записи о тех заказах, которые были возвращены - номер заказа, регион и статус возврата.

Довольно необычной особенностью этого набора данных является то, что номера заказов в нем не уникальны. Заказы с одинаковым номером могли быть зарегистрированы в разных регионах:

In [20]:
(orders[['order_id', 'region']]
 .groupby('order_id',)
 .filter(lambda grp: grp['region'].nunique() >1)
 .reset_index(drop=True))

,order_id,region
0,CA-2015-AG10330140-42361,Western US
1,CA-2015-AG10330140-42361,Central US
2,CA-2015-AG10330140-42361,Central US
3,CA-2015-AG10330140-42361,Central US
4,CA-2014-BW11200140-41940,Eastern US
5,CA-2014-BW11200140-41940,Southern US
6,CA-2014-BW11200140-41940,Eastern US
7,CA-2014-BW11200140-41940,Eastern US
8,CA-2014-BW11200140-41940,Eastern US
9,CA-2012-CS12355140-41135,Southern US


По этой причине соединять таблицы необходимо по двум ключам - order_id и region.

Попробуем соединить таблицы:

In [21]:
orders_returns = orders.merge(returns, on=['order_id', 'region']) # лучше явно указать ключи

In [22]:
orders_returns.shape

(2202, 29)

Строк получилось очень мало. В исходном наборе было более 50 тысяч строк:

In [23]:
orders.shape

(51290, 28)

## 2.1. Почему пропали строки?

Исправьте аргументы merge(), чтобы задать правильный тип соединения.

In [24]:
orders_returns = orders.merge(returns, on=['order_id', 'region'], how='left')


In [25]:
orders_returns.shape

(51290, 29)

In [26]:
orders_returns['returned'].fillna(False, inplace=True)
orders_returns

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,postal_code,city,...,quantity,discount,profit,shipping_cost,order_priority,share_delcost_sales,share_delcost_sales_mean,dif_share,order_year,returned
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120,Oklahoma City,...,2,0.0,62.1544,40.770,High,0.183665,0.184264,-0.000599,2014,False
1,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,...,9,0.1,-288.7650,923.630,Critical,0.248997,0.125315,0.123682,2014,False
2,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,...,9,0.1,919.9710,915.490,Medium,0.176900,0.172136,0.004765,2014,False
3,13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,...,5,0.1,-96.5400,910.160,Medium,0.314661,0.183456,0.131205,2014,False
4,47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,...,8,0.0,311.5200,903.040,Critical,0.318762,0.153027,0.165735,2014,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2015-KE1642066-42174,2015-06-19,2015-06-19,Same Day,KE-1642066,Katrina Edelman,Corporate,NaN,Kure,...,5,0.0,4.5000,1.010,Medium,0.015515,0.172017,-0.156503,2015,False
51286,34337,US-2014-ZD21925140-41765,2014-05-06,2014-05-10,Standard Class,ZD-219251408,Zuschuss Donatelli,Consumer,37421,Chattanooga,...,5,0.2,3.3440,1.930,High,0.115431,0.084991,0.030440,2014,False
51287,31315,CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,Second Class,ZD-219251404,Zuschuss Donatelli,Consumer,94109,San Francisco,...,2,0.0,2.4824,1.580,High,0.184579,0.150292,0.034287,2012,False
51288,9596,MX-2013-RB1979518-41322,2013-02-17,2013-02-21,Standard Class,RB-1979518,Ross Baird,Home Office,NaN,Valinhos,...,2,0.0,2.4000,1.003,Medium,0.074628,0.103933,-0.029305,2013,False


## 2.2. Составьте рейтинг регионов по убыванию доли возвращенных заказов.

In [27]:
regions_returns = orders_returns.groupby(['region', 'returned']).size().unstack().fillna(0)
regions_returns['share_returns'] = regions_returns[1] / (regions_returns[0] + regions_returns[1])
regions_returns.sort_values('share_returns', ascending=False)

returned,False,True,share_returns
region,,,
Western US,3026.0,177.0,0.055261
Eastern Asia,2243.0,131.0,0.055181
Southern Europe,2001.0,112.0,0.053005
Southern Africa,453.0,25.0,0.052301
Southern US,1537.0,83.0,0.051235
Eastern US,2714.0,134.0,0.047051
Southeastern Asia,2989.0,140.0,0.044743
South America,2855.0,133.0,0.044511
Western Asia,2332.0,108.0,0.044262


## 2.3. Рейтинг менеджеров

В еще одной таблице - people - содержатся данные о менеджерах, которые руководят продажами в различных регионах:

In [28]:
people = pd.read_csv('../datasets/global_superstore/people.csv')
people.head()


,person,region
0,Marilène Rousseau,Caribbean
1,Andile Ihejirika,Central Africa
2,Nicodemo Bautista,Central America
3,Cansu Peynirci,Central Asia
4,Lon Bonher,Central US


Соедините таблицу orders с этими данными и выведите рейтинг менеджеров по прибыли.

In [29]:
orders_managers = orders.merge(people, on='region', how='left')
orders_managers[['person', 'profit']].groupby('person').sum().sort_values('profit', ascending=False)

,profit
person,
Gilbert Wolff,218433.50850
Hadia Bousaid,167101.85100
Chandrakant Chaudhri,159336.42700
Nicodemo Bautista,158981.64816
Kauri Anaru,120089.11200
Derrick Snyders,108418.44890
Dolores Davis,91522.78000
Miina Nylund,83923.91700
Oxana Lagunov,77084.88000


## 2.4. Объединение таблиц

На основе таблицы orders определите три самые прибыльные товарные подкатегории на рынке США и сохраните их в таблице best_usa. Затем на основе этой же таблицы определите три самые прибыльные подкатегории на рынке Европы и сохраните их в таблице best_europe. Затем объедините данные из таблиц best_usa и best_europe в одной таблице.

In [30]:
markets = orders[['market', 'sub_category', 'profit']].groupby('market')
best_usa = None
best_europe = None
for name, data in markets:
    if name == "USCA":
        best_usa = data
    
    if name == "Europe":
        best_europe = data
        
best_usa = (
            best_usa
                .groupby('sub_category')
                .sum()
                .sort_values('profit', ascending=False)
                .head(3)
           )
best_usa

,profit
sub_category,
Copiers,58281.4649
Phones,47195.6006
Accessories,43231.9457


In [31]:
best_europe = (
            best_europe
                .groupby('sub_category')
                .sum()
                .sort_values('profit', ascending=False)
                .head(3)
           )
best_europe

,profit
sub_category,
Bookcases,68138.595
Copiers,62994.864
Appliances,52963.110


In [32]:
merged = pd.concat([best_usa, best_europe], sort=False)
merged

,profit
sub_category,
Copiers,58281.4649
Phones,47195.6006
Accessories,43231.9457
Bookcases,68138.5950
Copiers,62994.8640
Appliances,52963.1100
